In [1]:
from __future__ import division
import numpy as np
import math
import pandas as pd
from collections import Counter
import random

In [2]:
#reading data
yx = []
x = []
full_text = []
with open('DocumentClassificationTrainingData.txt') as fi:
    num = fi.readline()
    for line in fi:
        line = line.replace('\n','')
        data = line.strip().split(' ')
        yx.append(int(data[0]))
        x.append(Counter(data[1:]))
    fi.close()

In [3]:
frame = pd.DataFrame(x)

In [4]:
frame['label'] = yx

In [5]:
binary_frame = frame[(frame['label']==1)|
                     (frame['label']==2)]

In [6]:
features = frame.columns[:-1]

In [7]:
binary_frame=binary_frame.fillna(0)

In [8]:
hxtrain = binary_frame[features].as_matrix()[:-100]
yxtrain = binary_frame['label'].as_matrix()[:-100]

In [14]:
hxtest = binary_frame[features].as_matrix()[-100:]
yxtest = binary_frame['label'].as_matrix()[-100:]
hxtrain2 = hxtrain[:50]
yxtrain2 = yxtrain[:50]

In [15]:
W = np.random.rand(1,len(features))[0]

In [16]:
def logistic(x):
    return 1 / (1+math.exp(-x))

In [17]:
def getProbabilitie(X,W):
    y_h=[logistic(np.dot(x_i,W)) for x_i in X]
    return y_h

In [18]:
def getIntBool(y):
    if(y==+1):
        return 1
    else:
        return 0

In [19]:
def getErrorIndicator(Y,Y_h):
    indicator = map(getIntBool, Y)
    error = [(ind_i - y_h) for ind_i,y_h in zip(indicator,Y_h)]
    return error

In [20]:
def getDerivative_j(error,hx_feature):
    return np.dot(error,hx_feature)

In [21]:
def getGradient(error,hx,W):
    gradient = np.zeros(len(W))
    for j,_ in enumerate(W):
        gradient[j] = getDerivative_j(error,hx[:,j])
    return gradient

In [22]:
def updateCoefficient(W,gradient,step_size):
    W = W + step_size*gradient
    return W

In [23]:
def getLogLikelihood_i(yi,y_hi):
    ly_hi = math.log(y_hi)
    if(yi == 1):
        return  ly_hi
    else:
        return (1-ly_hi)

In [24]:
y_h = getProbabilitie(hxtrain2,W)

In [25]:
ll = sum(getLogLikelihood_i(y_i,yh_i) for y_i,yh_i in zip(yxtrain2,y_h)) 

In [26]:
print "log likelihood: " + str(ll)

log likelihood: 8.98749788938


In [27]:
error = getErrorIndicator(yxtrain2,y_h)

In [28]:
gradient = getGradient(error,hxtrain2,W)

In [29]:
W = updateCoefficient(W,gradient,0.1)

In [30]:
y_h = getProbabilitie(hxtrain2,W)

In [31]:
ll = sum(getLogLikelihood_i(y_i,yh_i) for y_i,yh_i in zip(yxtrain2,y_h)) 

In [32]:
print "log likelihood: " + str(ll)

log likelihood: 245.176636528


In [60]:
for i in xrange(1):
    y_h = getProbabilitie(hxtrain2,W)
    error = getErrorIndicator(yxtrain2,y_h)
    gradient = getGradient(error,hxtrain2,W)
    W = updateCoefficient(W,gradient,0.1)
    ll = sum(getLogLikelihood_i(y_i,yh_i) for y_i,yh_i in zip(yxtrain2,y_h))
    print "log likelihood: " + str(ll) + " i= " + str(i)

log likelihood: 1053.3616808 i= 0


In [56]:
yx_indicator = map(getIntBool,yxtrain2)

In [57]:
y_h_disc = [round(y_hi) for y_hi in y_h]

In [58]:
error_class = sum([(y-yh) for y,yh in zip(yx_indicator,y_h_disc)])

In [59]:
error_class

0.0